<a href="https://www.kaggle.com/code/dianasivkova/notebook884a056823?scriptVersionId=217108505" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Цель и описание проекта

Цель данного проекта - создание модели для поиска изображений по текстовым запросам, которая способна оценивать соответствие текста и изображения. При этом необходимо учитывать юридические ограничения: изображения, содержащие детей, должны быть исключены из результатов поиска. Наш целевой признак - вероятность соответствия текста и изображения - будет принимать значения от 0 до 1.

Перед началом исследования мы загрузим и прочитаем весь необходимый набор данных - тренировочную и тестовую выборку и таблицы с краудсорсинговыми и экспертными оценками.

Мы проведем исследовательский анализ тренировочной выборки и выясним, какие слова чаще всего встречаются в текстовых запросах. Анализируя датасеты с краудсорсинговыми и экспертными оценками, мы изобразим на графиках самые часто встречающиеся оценки и долю верных описаний. Мы создадим столбцы с  вероятностью соответствия текста к картинке на основе краудсорсинговых оценок и с агрегированными экспертными оценками, которая будет вычислятся "большинством". Далее мы объединим эти две оценки в одну - используя разные коэффициенты для краудсорсинговых и экспертных оценок - и присоединим финальную оценку к тренировочному датасету по query_id.

Следующим нашим шагом будет создание списка слов, связанных с описанием детей, и функции, которая удалит все текстовые запросы с словами, которые встречаются в этом списке.

Следующим этапом будет векторизация изображений и текстов. Для векторизации изображений мы будем использовать сверточную сеть ResNet50, а для векторизации текста - TF IDF. Далее мы объединим эти вектора в финальный датасет, который будет использоваться для обучения.

Для предсказания нашего целевого признака будут использованы нейронная сеть и регрессионные модели - DummyRegressor, Ridge, RandomForestRegressor. В качестве метрики мы выберем MeanAbsoluteError. Модель с самой низкой метрикой будет использована для предсказания вероятности соответствия текста и изображения на тестовой выборке.

Для предсказания целевого признака на тестовых данных, мы создадим эмбеддинги всех изображений в тестовой выборке, и выберем случайным образом десять текстовых запросов. Далее мы выведем наиболее подходящее изображение для каждого запроса, при этом текстовые запросы, содержащие слова, связанные с описанием детей, обрабатываться не будут - вместо изображения будет показан дисклеймер, информирующий о недоступности изображения в соответствии с законодательными ограничениями.

В конце исследования будет сделан вывод о точности созданной модели. Мы оценим её способность находить релевантные изображения по текстовым запросам.

## Загрузка и исследовательский анализ данных


In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from collections import Counter
import string
import seaborn as sns
import os
import random

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GroupShuffleSplit

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
main_path = '/kaggle/input/images'
train_images_dir = '/kaggle/input/train-images'
test_images_dir = '/kaggle/input/test-images-all'
stopwords = stopwords.words('english')
RANDOM_STATE = 42

Загрузим и прочитаем все датасеты.

In [4]:
df_train = pd.read_csv(Path(main_path, 'train_dataset.csv'))
df_crowd = pd.read_csv(Path(main_path, 'CrowdAnnotations.tsv'), sep='\t',
                       names=['image', 'query_id', 'share_pos', 'num_pos', 'num_neg'])
df_expert = pd.read_csv(Path(main_path, 'ExpertAnnotations.tsv'), sep='\t',
                        names=['image', 'query_id', 'first_expert',
                               'second_expert', 'third_expert'])
df_test = pd.read_csv(Path(main_path, 'test_queries.csv'), index_col=[0], sep='|')
df_test_images = pd.read_csv(Path(main_path, 'test_images.csv'), sep='|')

In [5]:
df_train.head(10)

,image,query_id,query_text
0,1056338697_4f7d7ce270.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
1,1262583859_653f1469a9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
2,2447284966_d6bbdb4b6e.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
3,2549968784_39bfbe44f9.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
4,2621415349_ef1a7e73be.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
5,3030566410_393c36a6c5.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
6,3155451946_c0862c70cb.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
7,3222041930_f642f49d28.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
8,343218198_1ca90e0734.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...
9,3718964174_cb2dc1615e.jpg,2549968784_39bfbe44f9.jpg#2,A young child is wearing blue goggles and sitt...


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5822 entries, 0 to 5821
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image       5822 non-null   object
 1   query_id    5822 non-null   object
 2   query_text  5822 non-null   object
dtypes: object(3)
memory usage: 136.6+ KB


In [ ]:
print(f"Кол-во уникальных картинок в тренировочном датасете {df_train['image'].nunique()}")
print(f"Кол-во уникальных описаний в тренировочном датасете {df_train['query_id'].nunique()}")

Тренировочные данные содержат 5822 строк с 1000 уникальными изображениями и 977 уникальными описаниями. Каждая картинка может иметь до пяти описаний.

Следующим нашим шагом будет создание графика с самыми популярными словами в описаниях, исключая стоп слова.

In [ ]:
def clean_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()
    words = [word for word in text.split() if word not in stopwords]
    return words

In [ ]:
all_words = []
df_train['query_text'].apply(lambda x: all_words.extend(clean_text(x)))
word_counts = Counter(all_words)

In [ ]:
most_common_words = word_counts.most_common(10)

words, counts = zip(*most_common_words)

plt.figure(figsize=(10, 6))
plt.bar(words, counts, color='pink')
plt.title('Десять самых популярных слов в описаниях фотографий')
plt.xlabel('Слова')
plt.ylabel('Частота')
plt.tight_layout()
plt.show()

В нашей тренировочной выборке представлено больше всего фотографий с собаками и мужчинами. Около 1000 фотографий содержат в описании детей - эти фотографии необходимо будет исключить из обучения модели.

В файле CrowdAnnotations.tsv — данные по соответствию изображения и описания, полученные с помощью краудсорсинга. Номера колонок и соответствующий тип данных:

Имя файла изображения.

Идентификатор описания.

Доля людей, подтвердивших, что описание соответствует изображению.

Количество человек, подтвердивших, что описание соответствует изображению.

Количество человек, подтвердивших, что описание не соответствует изображению.

In [ ]:
df_crowd.info()

In [ ]:
df_crowd.head(10)

В данных с краудсорсинговыми оценками мы видим 47 830 строк без пропусков.

Создадим график с изображением доли верных и неверных описаний в датасете с краудсорсинговыми оценками.

In [ ]:
df_crowd['correct_description'] = df_crowd['num_pos'] > df_crowd['num_neg']

correct_images = df_crowd[df_crowd['correct_description']]['image'].nunique()  
incorrect_images = df_crowd[~df_crowd['correct_description']]['image'].nunique() 

data = {'Верное описание': correct_images, 'Неверное описание': incorrect_images}

plt.figure(figsize=(8, 6))
plt.bar(data.keys(), data.values(), color=["green", "purple"])
plt.xlabel('Тип описания', fontsize=12)
plt.ylabel('Кол-во изображений', fontsize=12)
plt.title('Кол-во изображений с верным и неверным описанием', fontsize=16)

plt.show()

Большинство картинок в датасете имеют неверное описание. Можно сделать вывод, что при прогнозировании вероятности соответствия описания к изображению, мы не можем полагаться только на краудсорсинговые оценки.

Было принято решение создать столбец с вероятностью соответствия текста к картинке на основе краудсорсинговых оценок с помощью функции. Мы также удалим строки с недостаточным количеством голосов.

In [ ]:
def calculate_crowd_probability(row):

    total_votes = row['num_pos'] + row['num_neg']
    if total_votes == 0 or total_votes < 2:
        return None
    return row['num_pos'] / total_votes

df_crowd['crowd_probability'] = df_crowd.apply(calculate_crowd_probability, axis=1)

df_crowd = df_crowd.dropna(subset=['crowd_probability'])

In [ ]:
df_crowd.head(5)

In [ ]:
df_expert.info()

In [ ]:
df_expert.head(10)

В датасете с экспертными оценками представлены 5822 оценки от трех разных экспертов. Пропуски отсутствуют.

In [ ]:
df_expert['average_score'] = df_expert[['first_expert',
                                        'second_expert', 'third_expert']].mean(axis=1)

In [ ]:
plt.figure(figsize=(10, 6))
sns.kdeplot(df_expert['average_score'], fill=True, color='green', bw_adjust=1.5)
plt.title("Плотность распределения средней оценки для изображений", fontsize=16)
plt.xlabel("Средняя оценка", fontsize=12)
plt.ylabel("Плотность", fontsize=12)
plt.tight_layout()
plt.show();

Построив график с средними оценками соответствия изображения и запроса. Мы видим,что подавляющее большинство пар изображения-запрос не соответствуют друг другу, либо запрос содержит элементы описания изображения, но в целом запрос тексту не соответствует. В датасете представлено совсем немного пар, где запрос и текст соответствуют полностью.

С помощью функции создадим столбец с агрегированными оценками, которая вычисляется "большинством" - если хотя бы одна оценка встречается минимум дважды, считается, что у этой строки есть "большинство". Далее мы нормализуем данную оценку, чтобы она была в диапазоне от 0 до 1.

Если явного большинства нет, то возвращается None. Такие строки удаляются.

In [ ]:
def aggregate_scores(row):
    scores = [row['first_expert'], row['second_expert'], row['third_expert']]
    score_counts = pd.Series(scores).value_counts()
    if score_counts.max() >= 2:
        majority_score = score_counts.idxmax()
        return (majority_score - 1) / 3 #нормализация
    else:
        return None
df_expert['aggregated_score'] = df_expert.apply(aggregate_scores, axis=1)

#удаляем строки с none в столбце с агрегированной оценкой
df_expert = df_expert.dropna(subset=['aggregated_score'])

In [ ]:
df_expert.head(5)

In [ ]:
df_expert.info()

Мы создали столбец с агрегированной оценкой, исходя из самой популярной оценки среди трех экспертов. Строки, где у каждого эксперта были разные оценки (примерно 200), были удалены. 

In [ ]:
df_test.info()

In [ ]:
df_test.head(10)

In [ ]:
df_test_images.head(5)

In [ ]:
df_test_images.info()

Тестовая выборка содержит 100 изображений и 500 описаний.

In [ ]:
#создадим единую таблицу с краудсорсинговыми и экспертными оценками
merged_data = pd.merge(
    df_expert, df_crowd, 
    on=['image', 'query_id'], 
    how='outer'
)

In [ ]:
#напишем функцию, которая объединит экспертную и краудсортинговую оценки в одну
def unite_score(row):
    expert_score = row['aggregated_score']
    crowd_score = row['crowd_probability']
    if pd.isna(expert_score): 
        return crowd_score
    elif pd.isna(crowd_score):
        return expert_score
    return (0.6 * expert_score) + (0.4 * crowd_score)

merged_data['final_score'] = merged_data.apply(unite_score, axis=1)

In [ ]:
merged_data.info()

In [ ]:
merged_data.head(5)

Объеденим тренировочную выборку и датасет с финальной оценкой.

In [ ]:
df_train = pd.merge(df_train, merged_data[['image', 'query_id', 'final_score']],
                    how='outer', on=['image', 'query_id'])
unique_texts = df_train[['query_id', 'query_text']].dropna().drop_duplicates()
#переменная с уникальными описаниями

In [ ]:
#добавляем запросы обратно в df_train как уникальные значения на основе query_id

df_train = df_train.drop(columns=['query_text'], errors='ignore')
df_train = df_train.merge(unique_texts, on='query_id', how='left')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
print(f'Количество дубликатов - {df_train.duplicated().sum()}')

In [ ]:
df_train.isna().sum()

In [ ]:
#удаляем пропуски
df_train.dropna(subset=['final_score','query_text'],inplace=True)

In [ ]:
df_train.info()

Наша тренировочная выборка готова к обучению. Мы рассчитали целевую переменную (вероятность соответствия запроса к картинке) к каждой строке, объединили с тренировочными данными, убрали пропуски и проверили датасет на наличие дубликатов.

Создадим функцию для показа первых пяти фотографий из тренировочного и тестового датасета.

In [ ]:
def show_pic(df, path):
    sample = df.head(5)
    fig, axes = plt.subplots(1, len(sample), figsize=(15, 5))
    for idx, (i, row) in enumerate(sample.iterrows()):
        image_path = os.path.join(path, row['image'])
        img = Image.open(image_path)
        axes[idx].imshow(img)
        axes[idx].axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
show_pic(df_train, train_images_dir)

In [ ]:
show_pic(df_test_images, test_images_dir)

При просмотре изображений из тренировочной и тестовой выборки, можно заметить фотографии детей, которые будет необходимо удалить из выборки. Этим мы займемся в следующем шаге.

Мы загрузили и просмотрели все датасеты, провели исследовательский анализ тренировочной выборки - нашли самые используемые слова в описаниях, проанализировали, что самая частая краудсорсинговая оценка показывает, что картинка и запрос не соответствуют друг другу, в экспертных оценках также несоответствие встречается чаще.

Мы создали функцию, которые посчитали вероятность соответствия запроса и картинки на основе краудсорсинговых оценок, и функцию, которая агрегирует экспертные оценки на основе большинства. Далее мы объединили данные оценки в одну финальную - используя разные коэффициенты для краудсорсинговой и экспертной оценки.

Следующим нашим шагом было объединение тренировочной выборки и финальных оценок.

Проанализировав краудсорсинговые и экспертные оценки, в большинстве случаев было замечено несоответствие описания и изображения. Этот факт, скорее всего, негативно скажется на модели, которая не сможет достаточно хорошо прогнозировать соответствие запроса и изображения.

##  Проверка данных

В некоторых странах, где работает ваша компания, действуют ограничения по обработке изображений: поисковым сервисам и сервисам, предоставляющим возможность поиска, запрещено без разрешения родителей или законных представителей предоставлять любую информацию, в том числе, но не исключительно тексты, изображения, видео и аудио, содержащие описание, изображение или запись голоса детей. Ребёнком считается любой человек, не достигший 16 лет.
В вашем сервисе строго следуют законам стран, в которых работают. Поэтому при попытке посмотреть изображения, запрещённые законодательством, вместо картинок показывается дисклеймер:

⎪ This image is unavailable in your country in compliance with local laws

Однако у вас в PoC нет возможности воспользоваться данным функционалом. Поэтому все изображения, которые нарушают данный закон, нужно удалить из обучающей выборки.

Создадим список слов, которые указывают на содержание детей на изображении.

In [ ]:
underage_keywords = [
    "boy","girl", "girls","kids", "boys","child", "children", "kid", "kids", "minor", "teenager", "teen", "adolescent", "juvenile",
    "preteen", "underage", "infant", "toddler", "baby", "preschooler", "schoolchild", "young",
    "youth", "kid's", "child's", "under 18", "nursery", "daycare", "playground", "youngster",
    "pre-adolescent", "little", "prepubescent", "youthfulness", "early age"
]

Cоздадим функцию,которая отфильтрует наш датасет, удалив описания, в которых встречаются слова из underage_keywords.


In [ ]:
def contains_keywords(description):
    for word in underage_keywords:
        if word.lower() in description.lower():
            return True
    return False


df_train = df_train[~df_train['query_text'].apply(contains_keywords)]
df_train.reset_index(inplace=True,drop=True)

In [ ]:
df_train.info()

In [ ]:
print(f'Кол-во уникальных описаний {df_train["query_text"].nunique()}')

Количество уникальных описаний уменьшилось на 300, количество строк в датасете уменьшилось на 15 000.

## Векторизация изображений

Перейдём к векторизации изображений.
Самый примитивный способ — прочесть изображение и превратить полученную матрицу в вектор. Такой способ нам не подходит: длина векторов может быть сильно разной, так как размеры изображений разные. Поэтому стоит обратиться к свёрточным сетям: они позволяют «выделить» главные компоненты изображений. Как это сделать? Нужно выбрать какую-либо архитектуру, например ResNet-18, посмотреть на слои и исключить полносвязные слои, которые отвечают за конечное предсказание. При этом можно загрузить модель данной архитектуры, предварительно натренированную на датасете ImageNet.

In [ ]:
def load_train(path):
    train_datagen = ImageDataGenerator()

    train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory=path,
        x_col='image',
        y_col='final_score',
        target_size=(224, 224),
        batch_size=16,
        class_mode='raw',
        subset='training',
        shuffle = False,
        seed=12345)

    return train_gen_flow

In [ ]:
train_gen_flow = load_train(train_images_dir)

In [ ]:
def create_model(input_shape = (224,224,3)):
    backbone = ResNet50(input_shape=input_shape,
                    weights='imagenet',
                    include_top=False)
    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D())

    optimizer = Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='mean_squared_error',
                  metrics=['mae'])


    return model

In [ ]:
def vect_predict(model, df):
    predictions = model.predict(df)
    return predictions

In [ ]:
image_embeds = vect_predict(create_model(), train_gen_flow)

In [ ]:
image_embeds.shape

Мы получили эмбеддинги наших изображений.

## Векторизация текстов

Следующий этап — векторизация текстов. Вы можете поэкспериментировать с несколькими способами векторизации текстов:

tf-idf

word2vec

\*трансформеры (например Bert)

\* — если вы изучали трансформеры в спринте Машинное обучение для текстов.

In [ ]:
vectorizer = TfidfVectorizer(stop_words=stopwords)
text_embeds = (vectorizer.fit_transform(df_train['query_text'])).toarray()

In [ ]:
text_embeds.shape

Мы получили векторные представления для текстов.

## Объединение векторов

Подготовьте данные для обучения: объедините векторы изображений и векторы текстов с целевой переменной.

In [ ]:
#объединяем векторы изображений и текстов
combined_features = np.hstack([image_embeds, text_embeds])

In [ ]:
target_variable = df_train['final_score'].values

#финальный датасет с объединёнными признаками
final_data = pd.DataFrame(combined_features)
final_data['target'] = target_variable


In [ ]:
final_data.head(5)

In [ ]:
final_data.shape

In [ ]:
final_data.duplicated().sum()

Наша финальная тренировочная выборка состоит из 35 046 строк и 3157 признаков.

## Обучение модели предсказания соответствия

Для обучения разделите датасет на тренировочную и тестовую выборки. Простое случайное разбиение не подходит: нужно исключить попадание изображения и в обучающую, и в тестовую выборки.

 
Какую модель использовать — выберите самостоятельно. Также вам предстоит выбрать метрику качества либо реализовать свою.


Наш целевой признак - вероятность соответствия изображения тексту, которая принимает непрерывные значения от 0 до 1. Поэтому, мы будем решать задачу регресии.

Для оценки качества моделей была выбрана метрика MeanAbsoluteError, так как она ее легко интерпретировать и ее проще использовать для обучения.

Сначала мы обучим модель на DummyRegressor, чтобы посчитать метрику модели, которая предсказывает константу. Все дальнейшие метрики мы сравним с MAE у DummyRegressor, чтобы быть уверенными, что остальные модели действительно находят связи между входными и целевым признаками.

Нашими основными моделями будут Ridge, RandomForestRegressor и нейронная сеть.

In [ ]:
gss = GroupShuffleSplit(n_splits=1, train_size=.7, random_state=RANDOM_STATE)
train_indices, test_indices = next(gss.split(X=final_data.drop(columns=['target']),
                                             y=final_data['target'],
                                             groups=df_train['image']))
train_df, test_df = final_data.loc[train_indices], final_data.loc[test_indices]

In [ ]:
X_train = train_df.drop(columns=['target'])
y_train = train_df['target']

X_test = test_df.drop(columns=['target'])
y_test = test_df['target']

### Dummy, Ridge, RandomForest

Все наши признаки - количественные, поэтому перед обучением необходимо стандартизировать все входные признаки в тренировочной и тестовой выборке.

In [ ]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
dummy_model = DummyRegressor()
dummy_model.fit(X_train, y_train)
dummy_preds = dummy_model.predict(X_test)
dummy_mae = mean_absolute_error(y_test,dummy_preds)
print('MAE на Dummy модели', dummy_mae)

In [ ]:
ridge_model = Ridge(random_state = RANDOM_STATE, alpha  = 2)

In [ ]:
ridge_model.fit(X_train,y_train)

In [ ]:
ridge_preds= ridge_model.predict(X_test)
ridge_mae = mean_absolute_error(y_test,ridge_preds)
print(f'MAE Ridge {ridge_mae}')

In [ ]:
forest_model = RandomForestRegressor(random_state=RANDOM_STATE, max_depth = 17,n_estimators=5)

In [ ]:
forest_model.fit(X_train,y_train)

In [ ]:
forest_preds = forest_model.predict(X_test)
forest_mae = mean_absolute_error(y_test,forest_preds)
print(f'MAE RandomForestRegressor {forest_mae}')

MAE у DummyRegressor - модели, которая предсказывает константу, равен 0.14. Это значит, мы будем ориентироваться только на те модели, которые показывают метрику еще ниже.
В нашем случае MAE у Ridge и RandomForest равен 0.13 и 0.12 соответсвенно. Следующим шагом будет создание нейронной сети.

### Нейронная сеть

Нейронная сеть для решения нашей задачи представляет собой полносвязную сеть с тремя   слоями. Функция активации входного и скрытого слоя - ReLu, выходного - Sigmoid. Мы дважды используем Dropout для предотвращения переобучения.

В качестве оптимизатора мы выбрали Adam c установленной скоростью обучения 0.001.

В качестве функции потерь была выбрана MSE.

In [ ]:
nn_model = Sequential([
        Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')  
    ])

optimizer = Adam(learning_rate=0.0001)


nn_model.compile(optimizer=optimizer, loss=['mean_squared_error'],
            metrics=['mae'])

In [ ]:
nn_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size=32, epochs=20,
    verbose=2, shuffle=False
    )

In [ ]:
nn_preds = nn_model.predict(X_test)
nn_mae = mean_absolute_error(nn_preds, y_test)

In [ ]:
print(f"MAE у нейронной сети {nn_mae}")

In [ ]:
model_names = ['Dummy','Ridge','RandomForest','Neural Network']
mae_values = [dummy_mae, ridge_mae, forest_mae, nn_mae]
mae_df = pd.DataFrame({
    'Model': model_names,
    'MAE': mae_values
})
mae_df.sort_values(by = 'MAE')

Таблица с лучшей MSE метрикой каждой модели показывает, что нейронная сеть предсказывает целевой признак точнее всего. Именно эта модель будет использоваться для тестовой выборки.

На протяжении 20 эпох нейронная сеть показывала приблизительно один и тот же результат - метрика и лосс не менялись радикально.

## Тестирование модели

Настало время протестировать модель. Для этого получите эмбеддинги для всех тестовых изображений из папки test_images, выберите случайные 10 запросов из файла test_queries.csv и для каждого запроса выведите наиболее релевантное изображение. Сравните визуально качество поиска.


In [ ]:
def load_test(path):
    test_datagen = ImageDataGenerator()

    test_gen_flow = test_datagen.flow_from_dataframe(
        dataframe=df_test_images,
        directory=path,
        x_col='image',
        y_col=None,
        target_size=(224, 224),
        batch_size=16,
        class_mode='input',
        shuffle = False,
        seed=12345)

    return test_gen_flow

In [ ]:
test_gen_flow = load_test(test_images_dir)
test_image_embeds = vect_predict(create_model(),test_gen_flow)

In [ ]:
test_image_embeds.shape

Создадим функцию, которая выведет наиболее релевантное изображение для десяти случайных запросов. При запросе, описывающем детей, будет показано предупреждение

In [ ]:
def contains_underage_keywords(query, underage_keywords):
    for keyword in underage_keywords:
        if keyword.lower() in query.lower():
            return True
    return False

In [ ]:
def get_picture(text) -> None:
        if contains_underage_keywords(text, underage_keywords):
            print(text)
            print("Warning:  This image is unavailable in your country in compliance with local laws")
        else:
            text_embed = vectorizer.transform([text]).toarray() 
            X = np.concatenate(( 
                test_image_embeds,
                np.resize(text_embed, (test_image_embeds.shape[0],1108))),
                axis=1)
            X = scaler.transform(X)
            predictions = nn_model.predict(X)
            df = pd.concat((df_test_images, pd.Series(np.reshape(predictions, (predictions.shape[0],)), name='pred')), axis=1) #добавляем оценки к номерам картинок
            top = list(df.sort_values(by='pred', ascending=False)['image'].head(5))
            top_score = list(df.sort_values(by='pred', ascending=False)['pred'].head(5))
            print(text)
            fig = plt.figure(figsize=(15,5))
            plt.rcParams['axes.edgecolor'] = 'black'
            plt.rcParams['axes.linewidth'] = 0
            for i, (img, score) in enumerate(zip(top, top_score)):
                ax = fig.add_subplot(1, 6, i + 1)
                ax.imshow(Image.open(Path(test_images_dir, img)))
                ax.set_title(round(score, 2))
                ax.axis('off')
        
            if text in list(df_test['query_text']): #если текст был в исходном файле с описаниями - добавляем оригинальную картинку
                plt.rcParams['axes.edgecolor'] = 'green'
                plt.rcParams['axes.linewidth'] = 5
                fig.add_subplot(1, 6, 6)
                image = Image.open(Path(test_images_dir, df_test.iloc[df_test[df_test['query_text'] == text].index[0]]['image']))
                plt.imshow(image)
                plt.xticks([])
                plt.yticks([])
                plt.tight_layout()
        
            plt.show()

Комментарий студента

Честно признаюсь, я эту часть скопировала из интернета... 


In [ ]:
samples = df_test.sample(10)
list_samples = list(samples['query_text'])

In [ ]:
for text in list_samples:
    get_picture(text)

Наша модель отлично сопоставляет описание и изображение. При запросе всех картинок с детьми были показаны предупреждения.

Но, даже если модель в итоге правильно предсказывает изображение по запросу, на примерах можно увидеть, что модель ставит высокую вероятность соответствия (50-60%) текстового запроса (который описывает группу людей) изображению собак, или наоборот. 

В запросе "a snowboarder wearing a red jacket is going down a mountain" модель выделяет изображения, не соответствующие описанию, с высокими вероятностями.Часто вероятность распределяется слабо выраженным "лидером" среди изображений. Это может указывать на недостаточную уверенность модели в своих предсказаниях.

Модель показывает удовлетворительные результаты для простых запросов, но её точность и способность обрабатывать сложные текстовые описания можно значительно улучшить за счёт расширения тренировочных данных.



## Выводы



Целью проекта было создание модели, способной находить изображения по текстовым запросам, а также предсказывать вероятность соответствия текста и изображения с целевым значением от 0 до 1. Мы провели все этапы исследования, начиная от подготовки данных до обучения моделей и анализа их результатов.

Перед началом исследования загрузили тренировочные и тестовые выборки, а также таблицы с краудсорсинговыми и экспертными оценками. Мы провели исследовательский анализ тренировочного датасета: определили наиболее часто встречающиеся слова в текстах, выявили, что большинство изображений имеют неверное описание. Далее мы создали функцию для расчета вероятности соответствия текста и изображения на основе краудсорсинговых оценок и агрегировали экспертные оценки на основе большинства. Объединили данные оценки в одну финальную, используя разные коэффициенты для краудсорсинговых и экспертных оценок, и присоединили эту оценку к тренировочной выборке. Мы также исключили изображения с детьми из тренировочной выборке в соответствии с законодательными требованиями.

Для векторизации изображений использовали сверточную сеть ResNet50, а для векторизации текстов применили TF-IDF. Объединили данные эмбеддинги в финальный датасет для обучения.

На шаге обучения моделей сначала мы обучили DummyRegressor как базовую модель, показавшую MAE 0.14. Далее мы Ridge и RandomForestRegressor, которые продемонстрировали метрику MAE 0.13 и 0.12 соответсвенно. Мы создали нейронную сеть, которая показала лучшие результаты по метрике MAE среди всех моделей - 0.08. Эта модель была выбрана для тестовой выборки.

Во время предсказания целевого признака на тестовой модели нейронная сеть корректно обработала запросы, содержащие слова, связанные с детьми, выдав дисклеймер вместо изображений. По запросам с простым описанием модель смогла находить релевантные изображения, но для сложных запросов её точность снижалась.

Выводы по модели:
* Модель хорошо справляется с простыми текстовыми запросами, в которых описаны конкретные объекты или действия.
* Для сложных запросов с обилием деталей или контекстом модель часто присваивает высокую вероятность нерелевантным изображениям, что указывает на её недостаточную способность учитывать контекст.
* Среднее значение вероятностей у всех изображений часто распределяется равномерно, что демонстрирует низкую уверенность модели в своих предсказаниях.

Рекомендуется добавить изображения и текстовые описания с более сложными и контекстными примерами, чтобы улучшить способность модели обрабатывать разнообразные запросы.
